# Week 1 Optimisation

General Theory: For my team, maximise average points scored last year as well as points scored in that race last year

In [205]:
import requests
import pandas as pd
from pulp import *
import re

## Get Data

Get Cost Data

In [20]:
r = requests.get('https://fantasy-api.formula1.com/partner_games/f1/players')

In [33]:
cost = pd.DataFrame(r.json()['players'] )
cost = cost.set_index('id').reset_index(drop=True)

In [35]:
cost.head(2)

,first_name,last_name,team_name,position,position_id,position_abbreviation,price,current_price_change_info,status,injured,...,season_score,driver_data,constructor_data,born_at,season_prices,price_change,num_fixtures_in_gameweek,has_fixture,display_name,external_id
0,Scuderia Ferrari Mission Winnow,None,Scuderia Ferrari Mission Winnow,Constructor,2,CR,27.4,None,None,False,...,0,None,"{'best_finish': 1, 'best_finish_count': 239, '...",None,None,27.4,2,True,Ferrari,526:59:FER
1,ROKiT Williams Racing,None,ROKiT Williams Racing,Constructor,2,CR,6.5,None,None,False,...,0,None,"{'best_finish': 1, 'best_finish_count': 114, '...",None,None,6.5,2,True,Williams,481:172:WIL


Get 2019 fantasy points data and merge with cost data to get final data set for optimisation

In [49]:
driver = pd.read_pickle('../2019_Driver_Points.pickle')

In [57]:
constructor = pd.read_pickle('../2019_Constructor_Points.pickle')

In [76]:
cost.loc[cost.position=='Driver', 'display_name']=cost[cost.position=='Driver']['display_name'].str.replace(' ', '')

In [149]:
driver_data = cost[['display_name','position','price']][cost.position=='Driver'].merge(
    driver[['Name', 'AvgPoints', 'Mel']],
    left_on = 'display_name',
    right_on = 'Name',
    how='left'
).drop(columns=['Name'])

In [150]:
driver_data['AvgPoints'] = driver_data.AvgPoints.fillna(0)
driver_data['Mel'] = driver_data.Mel.fillna(0)

In [151]:
driver_data.head(2)

,display_name,position,price,AvgPoints,Mel
0,L.Hamilton,Driver,31.3,40.05,32.0
1,V.Bottas,Driver,28.4,31.48,48.0


In [152]:
constructor_data = cost[['display_name', 'position','team_abbreviation', 'price']][cost.position=='Constructor'].merge(
    constructor[['Constructor','team_abbreviation', 'AverageCon', 'Mel']],
    on='team_abbreviation',
    how='left'
).drop(columns=['Constructor', 'team_abbreviation']).rename(
    columns={
        'AverageCon': 'AvgPoints',
    }
)

In [153]:
constructor_data['AvgPoints'] = constructor_data.AvgPoints.fillna(0)
constructor_data['Mel'] = constructor_data.Mel.fillna(0)

In [154]:
constructor_data.head(2)

,display_name,position,price,AvgPoints,Mel
0,Ferrari,Constructor,27.4,45.8,42
1,Williams,Constructor,6.5,11.1,16


In [155]:
data = pd.concat([constructor_data, driver_data]).reset_index(drop=True)

In [156]:
data.head(2)

,display_name,position,price,AvgPoints,Mel
0,Ferrari,Constructor,27.4,45.8,42.0
1,Williams,Constructor,6.5,11.1,16.0


In [157]:
data.to_pickle('week1_data.pickle')

## Optimisation

Create Decison Variables

In [135]:
constructor_var = LpVariable.dict(
    'Constructor', 
    data['display_name'][data.position=='Constructor'], 
    0, 
    cat='Binary'
)

In [136]:
driver_var = LpVariable.dict(
    'Driver', 
    data['display_name'][data.position=='Driver'], 
    0, 
    cat='Binary'
)

Define Objective Function

In [189]:
prob = LpProblem('Fantasty_F1_Week_1', LpMaximize)

In [190]:
driver_rewards = []
constructor_rewards = []

In [191]:
list1 = list(data[data.position=='Driver'].eval('AvgPoints + 0.5*Mel'))
list2 = list(data[data.position=='Constructor'].eval('AvgPoints + 0.5*Mel'))
driver_rewards += lpSum( 
    [a*b for a, b in zip(list1, driver_var.values()) ]
)
constructor_rewards += lpSum( 
    [a*b for a, b in zip(list1, constructor_var.values()) ]
)

In [192]:
prob += driver_rewards + constructor_rewards, 'Total Points Scored'

Add constraints

In [193]:
prob += lpSum(driver_var) <= 5, 'Max Number of Drivers'
prob += lpSum(constructor_var) <= 1, 'Max Number of Constructor'

In [195]:
driver_costs = []
constructor_costs = []
list3 = list(data[data.position=='Driver'].price)
list4 = list(data[data.position=='Constructor'].price)

driver_costs += lpSum(
    [a*b for a,b in zip(list3, driver_var.values())]
)
constructor_costs += lpSum(
    [a*b for a,b in zip(list4, constructor_var.values())]
)

In [196]:
prob += driver_costs + constructor_costs <= 100, 'Total Cost of Team'

In [198]:
prob.solve()

1

In [199]:
print("Status:", LpStatus[prob.status])

Status: Optimal


### Results

In [202]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [206]:
summary (prob)

Variables:

Constructor_Williams = 1.0
Driver_G.Russell = 1.0
Driver_M.Verstappen = 1.0
Driver_P.Gasly = 1.0
Driver_S.Vettel = 1.0
Driver_V.Bottas = 1.0
---------------------------------------

Constraints:
6.5*1.0 + 5.9*1.0 + 26.1*1.0 + 10.4*1.0 + 21.8*1.0 + 28.4*1.0 = 99.1
---------------------------------------

Score:
55.480000000000004*1.0 + 15.31*1.0 + 46.55*1.0 + 22.81*1.0 + 37.55*1.0 + 55.480000000000004*1.0 = 233.18


## Optimisation - with Hamilton in the team

In [207]:
prob = LpProblem('Fantasty_F1_Week_1', LpMaximize)

In [208]:
#Objective Function
prob += driver_rewards + constructor_rewards, 'Total Points Scored'

In [211]:
#Constraints
prob += lpSum(driver_var) <= 5, 'Max Number of Drivers'
prob += lpSum(constructor_var) <= 1, 'Max Number of Constructor'
prob += driver_costs + constructor_costs <= 100, 'Total Cost of Team'
prob += driver_var['L.Hamilton'] == 1, 'Hamilton has to be in the team'

In [213]:
prob.solve()

1

In [214]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [215]:
summary (prob)

Variables:

Constructor_Williams = 1.0
Driver_G.Russell = 1.0
Driver_L.Hamilton = 1.0
Driver_M.Verstappen = 1.0
Driver_V.Bottas = 1.0
---------------------------------------

Constraints:
6.5*1.0 + 5.9*1.0 + 31.3*1.0 + 26.1*1.0 + 28.4*1.0 = 98.20000000000002
---------------------------------------

Score:
55.480000000000004*1.0 + 15.31*1.0 + 56.05*1.0 + 46.55*1.0 + 55.480000000000004*1.0 = 228.87


## Optimisation - with Hamilton & Mercedes in the team

In [217]:
prob = LpProblem('Fantasty_F1_Week_1', LpMaximize)

In [218]:
#Objective Function
prob += driver_rewards + constructor_rewards, 'Total Points Scored'

In [220]:
#Constraints
prob += lpSum(driver_var) <= 5, 'Max Number of Drivers'
prob += lpSum(constructor_var) <= 1, 'Max Number of Constructor'
prob += driver_costs + constructor_costs <= 100, 'Total Cost of Team'
prob += driver_var['L.Hamilton'] == 1, 'Hamilton has to be in the team'
prob += constructor_var['Mercedes'] == 1, 'Mercedes has to be in the team'

In [221]:
prob.solve()

1

In [222]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [223]:
summary (prob)

Variables:

Constructor_Mercedes = 1.0
Driver_D.Kvyat = 1.0
Driver_G.Russell = 1.0
Driver_K.Raikkonen = 1.0
Driver_L.Hamilton = 1.0
Driver_P.Gasly = 1.0
---------------------------------------

Constraints:
32.2*1.0 + 9.9*1.0 + 5.9*1.0 + 10.3*1.0 + 31.3*1.0 + 10.4*1.0 = 100.0
---------------------------------------

Score:
35.81*1.0 + 17.6*1.0 + 15.31*1.0 + 19.36*1.0 + 56.05*1.0 + 22.81*1.0 = 166.94


### END